<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/XGBoost_Recommender/Book_Crossing_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> XGBoost
 Recommender</font></h1>

---

<center>
  <img src="https://storage.googleapis.com/kaggle-datasets-images/1661575/2726067/684ac0c4c14cb46d1047ccb620b45cac/dataset-cover.jpg?t=2021-10-21-03-18-09" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en XGBoost.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

### 1. Contexto


El conjunto de datos "Book-Crossing" (también conocido como BX) es una colección de datos relacionados con libros y reseñas de libros. Este conjunto de datos se centra en la interacción de los usuarios con libros y sus calificaciones, y es ampliamente utilizado en aplicaciones de sistemas de recomendación.


<b>Descripción de datos</b>


El conjunto de datos Book-Crossing contiene información sobre:

---


<b>Usuarios (BX-Users):</b>

Contiene la información del usuario. Los campos incluyen:

* User-ID: Un identificador único para cada usuario.
* Location: La ubicación del usuario.
* Age: La edad del usuario.




<b>Libros (BX-Books):</b>

Contiene la información de los libros. Los campos incluyen:

* ISBN: Número de ISBN del libro, que es un identificador único.
* Book-Title: El título del libro.
* Book-Author: El autor del libro.
* Year-Of-Publication: El año de publicación del libro.
* Publisher: El editor del libro.
* Otras informaciones adicionales sobre los libros.




<b>Evaluaciones (BX-Book-Ratings):</b>

Contiene las evaluaciones de los libros. Los campos incluyen:

* User-ID: El identificador del usuario que dio la evaluación.
* ISBN: El ISBN del libro evaluado.
* Book-Rating: La calificación del libro en una escala (por lo general, de 1 a 10).




---



<strong>Puede consultar este [link](https://www.kaggle.com/datasets/syedjaferk/book-crossing-dataset) para leer más sobre la fuente de datos Book Crossing.</strong>

### 2. Descargar y preparar Dataset

In [ ]:
# Download Book-Crossing Dataset
!curl -o dataset.zip "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"
!unzip dataset.zip
!ls -la

In [ ]:
!curl -L -o dataset.zip "https://drive.google.com/uc?id=1P7_nW6mZAVgf7sDqdm3SaR9_ZaCOeyjI&export=download&authuser=0"
!unzip dataset.zip
!ls -la

In [11]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [ ]:
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding="ISO-8859-1")
books   = pd.read_csv("BX-Books.csv",        sep=";", encoding="ISO-8859-1", error_bad_lines=False)
users   = pd.read_csv("BX-Users.csv",        sep=";", encoding="ISO-8859-1")

In [ ]:
users.head()

In [ ]:
books.head()

<b>Calificaciones explícitas</b>: Están expresadas en una escala del 1-10 (más alta) y representan una calificación explícita por parte del usuario.

<b>Calificaciones implícitas</b>: Son expresadas por un 0, indicando que no hay una calificación explícita. En el contexto de este dataset, una calificación de 0 indica una interacción implícita con el libro (por ejemplo, el usuario lo compró o leyó), pero no proporciona una calificación explícita del contenido.

In [ ]:
ratings.head()

In [ ]:
print("  Users: {} \n  Books: {}\n  Ratings: {}".format(len(users), len(books), len(ratings)))


In [16]:
users.columns = users.columns.str.lower().str.replace('-', '_')
books.columns = books.columns.str.lower().str.replace('-', '_')
ratings.columns = ratings.columns.str.lower().str.replace('-', '_')

### 3. Uniendo data

In [17]:
# Analizaremos únicamente los datos explicitos del usuario-item
ratings = ratings[ratings.book_rating > 0]

In [ ]:
ratings.head()

In [ ]:
# Cruzamos las bases de datos para  obtener una tabla única

data = pd.merge(ratings, users, on = 'user_id', how = 'left')
data = pd.merge(data,    books, on = 'isbn', how = 'left')
data.drop(columns = ['image_url_s', 'image_url_m', 'image_url_l'], inplace = True)

data.head()

In [ ]:
# Estilo de Seaborn
sns.set(style="whitegrid")
# figura y eje
plt.figure(figsize=(6, 3))
sns.histplot(data.book_rating, bins=30, kde=False, color="skyblue")

In [21]:
#tratando información del año de publicación
data.year_of_publication = pd.to_numeric(data.year_of_publication, errors='coerce')


In [22]:
# Ejemplo de remoción de outliers
lower_threshold = 1964
upper_threshold = 2004

data = data[(data['year_of_publication'] >= lower_threshold) & (data['year_of_publication'] <= upper_threshold)]
data.year_of_publication = data.year_of_publication.astype(int)

In [25]:
#Creando antiguedad del libro
data['antiguedad'] = 2008 - data.year_of_publication

In [ ]:
# Estilo de Seaborn
sns.set(style="whitegrid")

# figura y eje
plt.figure(figsize=(6, 3))

# histograma
sns.histplot(data.antiguedad, bins=30, kde=False, color="skyblue")

# título y etiquetas a los ejes
plt.title('Distribución de antiguedad', fontsize=12)
plt.xlabel('Antiguedad', fontsize=10)
plt.ylabel('Frecuencia', fontsize=10)

# Muestra el histograma
plt.show()

In [ ]:
books_list = data.groupby('book_title')['user_id'].count().reset_index()
books_list.sort_values(by = 'user_id', ascending = False, inplace = True)

print(f"{len(books_list)} libros diferentes, nos quedaremos con los más populares para no saturar nuestro Recsys")

In [ ]:
books_list

In [ ]:
books_list[:500]

In [29]:
# Calculamos los libros más populares
pop_books = books_list[:500].book_title.tolist()

In [31]:
data_v2 = data[data.book_title.isin(pop_books)]

In [ ]:
data_v2.head()

Dicotomizaremos la variable objetivo para que el modelo aprenda la probabilidad de que el cliente tenga afinidad con el libro. Esta estrategia es bastante utilizada en las aplicaciones de Recsys pero no olvidemos que también se puede apuntar a predecir directamente el rating del cliente.

In [34]:
data_v2['target'] = data_v2.book_rating.apply(lambda x: 1 if x > 7 else 0)

In [ ]:
data_v2.head()

In [ ]:
# figura y eje
plt.figure(figsize=(6, 3))
# Analizando el target
sns.countplot(x='target', data = data_v2, palette = 'hls')
plt.title('¿La data presenta desbalance?', fontsize=12)


### 4. Muestreo de datos

In [37]:
# Muestreo de data
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_v2,
                               stratify = data_v2.target, # Recuerda estratificar para evitar sesgos durante el muestreo
                               train_size = 0.6,
                               random_state = 123)

watch, test = train_test_split(test,
                               stratify = test.target, # Recuerda estratificar para evitar sesgos durante el muestreo
                               train_size = 0.5,
                               random_state = 123)

# El muestreo puede hacerse por cliente o por enmascaramiento como en anteriores ejercicios.

### 5. Tratamiento de variables

Variable de locacion

In [ ]:
train.head()

In [ ]:
temp = train.groupby('location')['user_id'].count().reset_index()
temp.sort_values(by = 'user_id', ascending = False)

In [39]:
# Función para extraer los n últimos elementos y unirlos con ','
def extract_last_n(location, n):
    parts = location.split(', ')
    return ', '.join(parts[-n:])

# Generar agregaciones
train['location_level2'] = train['location'].apply(lambda x: extract_last_n(x, 2))
train['location_level3'] = train['location'].apply(lambda x: extract_last_n(x, 1))

test['location_level2'] = test['location'].apply(lambda x: extract_last_n(x, 2))
test['location_level3'] = test['location'].apply(lambda x: extract_last_n(x, 1))

watch['location_level2'] = watch['location'].apply(lambda x: extract_last_n(x, 2))
watch['location_level3'] = watch['location'].apply(lambda x: extract_last_n(x, 1))

In [ ]:
train.head()

In [ ]:
temp = train.groupby('location_level2')['user_id'].count().reset_index()
temp = temp[temp.user_id > 30]
temp.sort_values(by = 'user_id', ascending = False)

In [ ]:
temp = train.groupby('location_level3')['user_id'].count().reset_index()
temp = temp[temp.user_id > 30]
temp.sort_values(by = 'user_id', ascending = False)

In [46]:
# Creando variable mixta de locacion
train['location_f'] = train.apply(lambda row: row['location_level2'] if row['location_level3'] == 'usa' else row['location_level3'], axis=1)
test['location_f']  = test.apply(lambda row: row['location_level2'] if row['location_level3'] == 'usa' else row['location_level3'], axis=1)
watch['location_f'] = watch.apply(lambda row: row['location_level2'] if row['location_level3'] == 'usa' else row['location_level3'], axis=1)


In [ ]:
train.head()

**Encoding**

El encoding de variables categóricas convierte las categorías de texto en números de una manera que puede ser utilizada de manera eficiente por los algoritmos de machine learning.


In [ ]:
train.head()

In [245]:
catergory_features = ['book_title', 'book_author', 'publisher', 'location_f']

In [246]:
%%capture
!pip3 install category_encoders

In [ ]:
# Aplicando category encoders
from category_encoders import TargetEncoder

encoder = TargetEncoder(handle_unknown = 'infrequent_if_exist',
                        handle_missing = 'value',
                        min_samples_leaf = 30)

encoder.fit(train[catergory_features].astype('category'), train['target'])


In [248]:
# Aplicando transformaciones sobre  variables

train[[x + '_coded' for x in catergory_features]] = encoder.transform(train[catergory_features].astype('category'))
test[[x + '_coded' for x in catergory_features]]  = encoder.transform(test[catergory_features].astype('category'))
watch[[x + '_coded' for x in catergory_features]] = encoder.transform(watch[catergory_features].astype('category'))


In [ ]:
train.head()

### 6. Modelamiento




In [251]:
import xgboost as xgb
from sklearn.metrics import *

In [250]:
features = ['age', 'antiguedad', 'book_title_coded', 'book_author_coded', 'publisher_coded', 'location_f_coded']

In [273]:
# Definimos los parámetros para el Grid Search

param_grid = {'objective': ['binary:logistic'],
              'booster' : ['gbtree'],
              'learning_rate': [0.01, 0.05, 0.1],
              'max_depth': [3, 5, 7],
              'colsample_bytree': [0.7, 1],
              'subsample': [0.7, 1]}


In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

# Crear clasificador
xgBoost = xgb.XGBClassifier(use_label_encoder=False, n_estimators = 500)


# Crear objeto GridSearchCV
grid_search = GridSearchCV(xgBoost,
                           param_grid,
                           scoring = make_scorer(auc),
                           cv = 3,  # Número de folds en la validación cruzada
                           verbose = 2,  # Verbosidad del output
                           n_jobs = -1  # Uso de todos los núcleos disponibles
                          )

# Realizar búsqueda de parámetros
grid_search.fit(train[features],
                train.target,
                early_stopping_rounds = 10,
                eval_metric = "auc",
                eval_set=[(watch[features], watch.target)],
                verbose = True)



In [ ]:
# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Si deseas, también puedes extraer y visualizar los mejores parámetros encontrados
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")


In [373]:
%%capture
!pip install --upgrade xgboost

In [ ]:
# Entrenando el modelo final

xgBoost = xgb.XGBClassifier(use_label_encoder=False,
                            n_estimators = 500, **best_params)

xgBoost.fit(train[features],
            train.target,
            early_stopping_rounds=10,
            eval_metric="auc",
            eval_set=[(train[features], train.target), (watch[features], watch.target)],
            verbose=True)


# Extraer los resultados de evaluación
results = xgBoost.evals_result()


In [ ]:
epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)

# Ajusta el tamaño
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Watch')

ax.set_ylim([0.6, 0.7])  # Para limitar la cantidad de epochs

ax.legend()
plt.ylabel('AUC')
plt.title('XGBoost AUC')
plt.show()

In [ ]:
# Definir tamaño
fig, ax = plt.subplots(figsize=(5, 3))

# Graficar la importancia de las variables
xgb.plot_importance(xgBoost, importance_type="total_gain", ax=ax, title="Feature Importance (Gain)", show_values=False)

# Mostrar el gráfico
plt.show()

### 7. Evaluación del modelo

In [277]:
from scipy.stats import ks_2samp

# Definir métricas adicionales
def gini(y_true, y_score):
    auc = roc_auc_score(y_true, y_score)
    return 2*auc - 1

def ks_statistic(y_true, y_score):
    return ks_2samp(y_score[y_true == 1], y_score[y_true == 0]).statistic

In [283]:
# predicción del modelo
train['prediction'] = xgBoost.predict_proba(train[features])[:, 1]
test['prediction']  = xgBoost.predict_proba(test[features])[:, 1]
watch['prediction'] = xgBoost.predict_proba(watch[features])[:, 1]


In [ ]:
results = pd.DataFrame(columns=['Metric', 'Train', 'Test', 'Watch'])

metrics = [
    ("Accuracy", accuracy_score),
    ("Precision", precision_score),
    ("Recall", recall_score),
    ("F1 Score", f1_score),
    ("AUC-ROC", roc_auc_score),
    ("Gini", gini),
    ("KS Statistic", ks_statistic),
    ("Jaccard", jaccard_score)
]

for metric_name, metric_func in metrics:
    if metric_name in ["Gini", "KS Statistic"]:  # Si la métrica requiere probabilidades
        train_score = metric_func(train['target'], train['prediction'])
        test_score = metric_func(test['target'], test['prediction'])
        watch_score = metric_func(watch['target'], watch['prediction'])

    else:  # Si la métrica se aplica a etiquetas
        train_score = metric_func(train['target'], train['prediction'].apply(lambda x: 1 if x > 0.5 else 0))
        test_score = metric_func(test['target'],   test['prediction'].apply(lambda x: 1 if x > 0.5 else 0))
        watch_score = metric_func(watch['target'], watch['prediction'].apply(lambda x: 1 if x > 0.5 else 0))

    results = results.append({
        'Metric': metric_name,
        'Train': train_score,
        'Test': test_score,
        'Watch': watch_score
    }, ignore_index=True)


pd.set_option('display.float_format', '{:.2f}'.format)

# Mostrar los resultados
results

### 8. Utilizando nuestro RecSys

In [ ]:
# Seleccionamos un cliente cualquiera
test[test.user_id == 16795].head()


In [ ]:
train[train.user_id == 16795].head()


In [ ]:
train[train.user_id == 16795][['user_id'] + features].head()

In [ ]:
df_user = test[test.user_id == 16795][['user_id', 'age', 'location_f', 'location_f_coded']].drop_duplicates()
df_user.head()

In [356]:
# Información de todos los ítems
df_books = train[['antiguedad',	'book_title',	'book_author',	'publisher', 'book_title_coded',	'book_author_coded',	'publisher_coded']].drop_duplicates()
df_books['user_id'] =  16795

In [ ]:

df_user_items = pd.merge(df_user, df_books, on = 'user_id', how = 'left')
df_user_items.head()


In [ ]:
df_user_items['prediction'] = xgBoost.predict_proba(df_user_items[features])[:, 1]
df_user_items = df_user_items.sort_values(by = 'prediction', ascending = False).drop_duplicates()
df_user_items.head()

In [ ]:
# Items que el sistema le recomienda
df_user_items[df_user_items.prediction > 0.5].head(10).book_title.tolist()

In [ ]:
# Items que vio
test[(test.user_id == 16795) & (test.book_rating > 6)].head(10).book_title.tolist()


In [ ]:
train[(train.user_id == 16795) & (train.book_rating > 6)].head(10).book_title.tolist()


---
## Gracias por completar este laboratorio!